# 深入刨析PyTorch的state_dict、parameters、modules源码

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_08_深入刨析PyTorch的state_dict、parameters、modules源码：
    
https://www.bilibili.com/video/BV12g411K7oq/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

save and load the model 官方文档：https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html

### vim编辑器
(查找函数时 下划线 _函数名 进行查找 得到高亮结果)

(:nohl 取消高亮）

## 模型保存

In [ ]:
import torch
import torchvision.models as models

model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

## 模型加载

In [ ]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

## 测试 Module API 功能

In [1]:
import torch

class Test(torch.nn.Module):
    def __init__(self):
        super(Test, self).__init__()
        self.linear1 = torch.nn.Linear(2, 3)
        self.linear2 = torch.nn.Linear(3, 4)
        self.batch_norm = torch.nn.BatchNorm2d(4)

### to函数修改数据类型

In [10]:
test_module = Test()
print(test_module._modules)
print(test_module._modules['linear1'])
print(test_module._modules['linear1'].weight.dtype)

OrderedDict([('linear1', Linear(in_features=2, out_features=3, bias=True)), ('linear2', Linear(in_features=3, out_features=4, bias=True)), ('batch_norm', BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))])
Linear(in_features=2, out_features=3, bias=True)
torch.float32


In [11]:
test_module.to(torch.double)
print(test_module._modules['linear1'].weight.dtype)

torch.float64


In [12]:
test_module.to(torch.float32)
print(test_module._modules['linear1'].weight.dtype)

torch.float32


### 查询模型参数

In [13]:
# 返回有序字典
print(test_module._modules)

OrderedDict([('linear1', Linear(in_features=2, out_features=3, bias=True)), ('linear2', Linear(in_features=3, out_features=4, bias=True)), ('batch_norm', BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))])


In [16]:
# 返回当前module的parameters字段
print(test_module._parameters)

OrderedDict()


In [17]:
# 返回当前module的buffers字段
print(test_module._buffers)

OrderedDict()


In [18]:
print(test_module.state_dict())

OrderedDict([('linear1.weight', tensor([[ 0.3908,  0.3170],
        [-0.5579,  0.5695],
        [-0.1297,  0.6104]])), ('linear1.bias', tensor([ 0.6716, -0.3837,  0.5175])), ('linear2.weight', tensor([[-0.3328, -0.1700, -0.2631],
        [-0.0073,  0.0994, -0.1933],
        [ 0.4597,  0.4848,  0.5522],
        [ 0.5448, -0.1338, -0.3027]])), ('linear2.bias', tensor([ 0.2380, -0.0048, -0.5631,  0.0804])), ('batch_norm.weight', tensor([1., 1., 1., 1.])), ('batch_norm.bias', tensor([0., 0., 0., 0.])), ('batch_norm.running_mean', tensor([0., 0., 0., 0.])), ('batch_norm.running_var', tensor([1., 1., 1., 1.])), ('batch_norm.num_batches_tracked', tensor(0))])


In [19]:
for p in test_module.parameters():
    print(p)

Parameter containing:
tensor([[ 0.3908,  0.3170],
        [-0.5579,  0.5695],
        [-0.1297,  0.6104]], requires_grad=True)
Parameter containing:
tensor([ 0.6716, -0.3837,  0.5175], requires_grad=True)
Parameter containing:
tensor([[-0.3328, -0.1700, -0.2631],
        [-0.0073,  0.0994, -0.1933],
        [ 0.4597,  0.4848,  0.5522],
        [ 0.5448, -0.1338, -0.3027]], requires_grad=True)
Parameter containing:
tensor([ 0.2380, -0.0048, -0.5631,  0.0804], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)


In [20]:
for p in test_module.named_parameters():
    print(p)

('linear1.weight', Parameter containing:
tensor([[ 0.3908,  0.3170],
        [-0.5579,  0.5695],
        [-0.1297,  0.6104]], requires_grad=True))
('linear1.bias', Parameter containing:
tensor([ 0.6716, -0.3837,  0.5175], requires_grad=True))
('linear2.weight', Parameter containing:
tensor([[-0.3328, -0.1700, -0.2631],
        [-0.0073,  0.0994, -0.1933],
        [ 0.4597,  0.4848,  0.5522],
        [ 0.5448, -0.1338, -0.3027]], requires_grad=True))
('linear2.bias', Parameter containing:
tensor([ 0.2380, -0.0048, -0.5631,  0.0804], requires_grad=True))
('batch_norm.weight', Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True))
('batch_norm.bias', Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True))


In [21]:
for p in test_module.named_children():
    print(p)

('linear1', Linear(in_features=2, out_features=3, bias=True))
('linear2', Linear(in_features=3, out_features=4, bias=True))
('batch_norm', BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))


In [22]:
for p in test_module.named_modules():
    print(p)

('', Test(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=4, bias=True)
  (batch_norm): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
))
('linear1', Linear(in_features=2, out_features=3, bias=True))
('linear2', Linear(in_features=3, out_features=4, bias=True))
('batch_norm', BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))


In [23]:
for p in test_module.modules():
    print(p)
    print('\n')

Test(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=4, bias=True)
  (batch_norm): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


Linear(in_features=2, out_features=3, bias=True)


Linear(in_features=3, out_features=4, bias=True)


BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


